In [ ]:
from gen_ai_hub.proxy.core.proxy_clients import set_proxy_version
from gen_ai_hub.proxy.langchain import init_llm


In [ ]:
from hana_ml import dataframe
from hana_ml.algorithms.pal.utility import DataSets
#from gen_ai_hub.proxy.langchain.init_models import init_llm
from hana_ai.tools.toolkit import HANAMLToolkit
from hana_ai.vectorstore.embedding_service import GenAIHubEmbeddings
from hana_ai.vectorstore.hana_vector_engine import HANAMLinVectorEngine
from hana_ai.vectorstore.code_templates import get_code_templates
llm = init_llm('gpt-4', temperature=0.0, max_tokens=512)

In [ ]:
url, port, user = "hana-ml-api.hana-ml.c.ap-cn-1.cloud.sap", 30015, "PAL_TEST"
connection_context = dataframe.ConnectionContext(url, port, user)
hana_df, _, _, _ = DataSets.load_iris_data(connection_context)
hana_df = hana_df.to_head("ID")

In [ ]:
embedding_func = GenAIHubEmbeddings()
toolkit = HANAMLToolkit(connection_context)
hana_vec = HANAMLinVectorEngine(connection_context, "hana_vec_hana_ml_knowledge")
hana_vec.create_knowledge()



In [ ]:
connection_context.table("hana_vec_hana_ml_knowledge").head(10).collect()

In [ ]:
toolkit.set_vectordb(hana_vec)

In [ ]:
from hana_ai.smart_dataframe import SmartDataFrame

sdf = SmartDataFrame(hana_df)

In [ ]:
sdf.configure(toolkit=toolkit, llm=llm)

In [ ]:
sdf.ask("Create dataset report", verbose=True)

In [ ]:
connection_context.drop_table("my_model")

In [ ]:
sdf.ask("Create Automatic Classification model on this dataframe with max_eval_time_mins=10. Provide key is ID, background_size is 100, label is SPECIES and model table is 'my_model' in the fit function and execute it. ", verbose=True)

In [ ]:
connection_context.table("my_model").collect()

In [ ]:
sdf.ask("Show the samples of the dataset", verbose=True)

In [ ]:
new_df = sdf.transform("Get first two rows", verbose=True)

In [ ]:
new_df.shape

In [ ]:
import pandas as pd
external_knowledge = get_code_templates(customized_dir="external_knowledge")

external_knowledge

In [ ]:
second_hana_vec = HANAMLinVectorEngine(connection_context, "second_hana_vec_hana_ml_knowledge")
second_hana_vec.upsert_knowledge(external_knowledge)

In [ ]:
# from hana_ai.vectorstore.union_vector_stores import merge_hana_vector_store

# cr = merge_hana_vector_store([hana_vec, second_hana_vec], table_name="merged_hana_vec_hana_ml_knowledge")

In [ ]:
from hana_ai.vectorstore.union_vector_stores import UnionVectorStores

uvs = UnionVectorStores([second_hana_vec, hana_vec])


In [ ]:
toolkit.set_vectordb(uvs)

In [ ]:
ts_df = DataSets.load_shampoo_data(connection_context)

In [ ]:
s_ts_df = SmartDataFrame(ts_df)
s_ts_df.configure(toolkit=toolkit, llm=llm)

In [ ]:
s_ts_df.head(1).collect()

In [ ]:
s_ts_df.ask("Create Automatic TimeSeires model on this dataframe. Provide key is ID, endog is SALES and model table is 'my_model_ts' in the fit function and execute it. ", verbose=True)

In [ ]:
connection_context.table("my_model_ts").collect()